In [1]:
import numpy as np
import pandas as pd

# Procesamos el Dataset en un Dataframe de Pandas

In [2]:
dataset = pd.read_excel("./src/shuttle-landing-control.ods")

print(dataset)

   Stability Error Sign  Wind   Magnitude Visibility   Class
0          *     *    *     *           *        yes    auto
1      xstab     *    *     *           *         no  noauto
2       stab    lx    *     *           *         no  noauto
3       stab    xl    *     *           *         no  noauto
4       stab    mm   nn  tail           *         no  noauto
5          *     *    *     *  OutOfRange         no  noauto
6       stab    ss    *     *         low         no    auto
7       stab    ss    *     *      medium         no    auto
8       stab    ss    *     *      strong         no    auto
9       stab    mm   pp  head         low         no    auto
10      stab    mm   pp  head      medium         no    auto
11      stab    mm   pp  tail         low         no    auto
12      stab    mm   pp  tail      medium         no    auto
13      stab    mm   pp  head      strong         no  noauto
14      stab    mm   pp  tail      strong         no    auto


### Antes de pasar a la metodologìa STAR damos tratamiento a los casos don't care (*)

In [3]:
from librerias_caseras.procesamiento_datasets import valores_observados, repeticiones_de_valor
import itertools    # <- https://blog.enterprisedna.co/how-to-generate-all-combinations-of-a-list-in-python/#:~:text=To%20generate%20all%20combinations%20of%20a%20list%20in%20Python%2C%20you,iterator%20algebra%20and%20combinatorial%20constructs.

def expandirTabla_desde_fila(fila:list[str], etiquetas_columnas:list[str], dataset:pd.DataFrame):
    '''Evalùa una fila del conjunto de datos, y si encuentra un * expande la tabla con todos los casos del dominio del atributo'''

    se_expande:bool = False
    contador:int = 0
    indices_de_expansion:list = []
    indices_ordinarios:list = []

    for i, valor_observado in enumerate(fila):
        if(valor_observado == '*'):
            se_expande = True
            contador += 1
            indices_de_expansion.append(i)
        else:
            contador += 1
            indices_ordinarios.append(i)

    if se_expande:

        dominios_atributos:list[list[str]] = []
        # observamos el dominio posible del atributo (en el preanálisis o bservamos que todos los valores del dominio de cada atributo aparece en su columna)
        for indice in range(len(fila)):

            if(indices_de_expansion.__contains__(indice)):

                dominios_atributos.append(valores_observados(valores_observados=dataset[etiquetas_columnas[indice]]))

            else:

                dominios_atributos.append([fila[indice]])

        print(dominios_atributos)
        
        for dominio_atributo in dominios_atributos:
            print(dominio_atributo)

            if(dominio_atributo.__contains__('*')):
                dominio_atributo.pop(dominio_atributo.index('*'))

        #ahora que tenemos todos los dominios de atributos, para ver que funciona, vamos a sacar todas las conbinaciones posibles con esos dimnios

        combinaciones:list[list[str]] = []

        for combinacion in itertools.product(*dominios_atributos):       #<- https://stackoverflow.com/questions/533905/how-to-get-the-cartesian-product-of-multiple-lists

            # print("\n\nlas conbinaciones de esos atributos: \n\n\n",combinacion)
            combinaciones.append(list(combinacion))

        print("\n\nlas conbinaciones de esos atributos: \n\n\n", np.matrix(combinaciones))

        return combinaciones
    
    else:

        return [fila]



In [4]:
filas_nuevo_dataset:list[list[str]] = []

# for row in dataset[dataset['Visibility'] == 'yes'].values.tolist():
for row in dataset.values.tolist():
    expansion_fila = expandirTabla_desde_fila(fila=row, etiquetas_columnas=dataset.columns.to_list(), dataset=dataset)

    for fila in expansion_fila:
        filas_nuevo_dataset.append(fila)

print("\n\nesto es lo que se contiene enla lista de filas nuevo dataset: \n\n", np.matrix(filas_nuevo_dataset))

[['*', 'xstab', 'stab'], ['*', 'lx', 'xl', 'mm', 'ss'], ['*', 'nn', 'pp'], ['*', 'tail', 'head'], ['*', 'OutOfRange', 'low', 'medium', 'strong'], ['yes'], ['auto']]
['*', 'xstab', 'stab']
['*', 'lx', 'xl', 'mm', 'ss']
['*', 'nn', 'pp']
['*', 'tail', 'head']
['*', 'OutOfRange', 'low', 'medium', 'strong']
['yes']
['auto']


las conbinaciones de esos atributos: 


 [['xstab' 'lx' 'nn' 'tail' 'OutOfRange' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'tail' 'low' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'tail' 'medium' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'tail' 'strong' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'head' 'OutOfRange' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'head' 'low' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'head' 'medium' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'head' 'strong' 'yes' 'auto']
 ['xstab' 'lx' 'pp' 'tail' 'OutOfRange' 'yes' 'auto']
 ['xstab' 'lx' 'pp' 'tail' 'low' 'yes' 'auto']
 ['xstab' 'lx' 'pp' 'tail' 'medium' 'yes' 'auto']
 ['xstab' 'lx' 'pp' 'tail' 'strong' 'yes' 'auto']
 ['xstab' 'lx' 'pp' 'head' 'OutO

### una vez que tenemos las filas de lo que sería la tabla expandida, procedemos a construir el nuevo dataframe con la información desarrollada

In [5]:
nuevo_dataset = pd.DataFrame(data=filas_nuevo_dataset, columns=dataset.columns)

print(nuevo_dataset)

    Stability Error Sign  Wind   Magnitude Visibility   Class
0       xstab    lx   nn  tail  OutOfRange        yes    auto
1       xstab    lx   nn  tail         low        yes    auto
2       xstab    lx   nn  tail      medium        yes    auto
3       xstab    lx   nn  tail      strong        yes    auto
4       xstab    lx   nn  head  OutOfRange        yes    auto
..        ...   ...  ...   ...         ...        ...     ...
273      stab    mm   pp  head      medium         no    auto
274      stab    mm   pp  tail         low         no    auto
275      stab    mm   pp  tail      medium         no    auto
276      stab    mm   pp  head      strong         no  noauto
277      stab    mm   pp  tail      strong         no    auto

[278 rows x 7 columns]


# Obtenemos una muestra proporcional 50 50 para entrenar

In [6]:
total_clases_original = []

clases = valores_observados(nuevo_dataset["Class"].values)
proporcion = []

for clase in clases:
    total = repeticiones_de_valor(clase, nuevo_dataset["Class"].values)
    total_clases_original.append([clase, total])

    proporcion.append([clase, total/len(nuevo_dataset["Class"].values)])

print(total_clases_original)
print(clases)
print(proporcion)

[['auto', 145], ['noauto', 133]]
['auto', 'noauto']
[['auto', 0.5215827338129496], ['noauto', 0.4784172661870504]]


In [7]:
from librerias_caseras.procesamiento_datasets import extraer_muestra

X_train, y_train, x_text, y_test = extraer_muestra(nuevo_dataset.drop("Class", axis=1), nuevo_dataset["Class"], proporcion, nuevo_dataset.__len__())

nuevo_dataset2 = pd.concat([X_train, y_train], axis=1)
test_set = pd.concat([x_text, y_test], axis=1)

print(nuevo_dataset2)

139
[['auto', 72], ['noauto', 66]]
['auto', 72]
145 145
['noauto', 66]
206 206
    Stability Error Sign  Wind   Magnitude Visibility   Class
0       xstab    lx   pp  tail      medium        yes    auto
1        stab    ss   nn  head      strong        yes    auto
2        stab    ss   pp  tail      medium         no    auto
3        stab    lx   pp  head         low        yes    auto
4       xstab    xl   nn  head      strong        yes    auto
..        ...   ...  ...   ...         ...        ...     ...
133     xstab    lx   nn  head  OutOfRange        yes    auto
134     xstab    mm   nn  tail      medium         no  noauto
135      stab    lx   pp  head  OutOfRange         no  noauto
136     xstab    lx   pp  head         low        yes    auto
137      stab    lx   nn  tail      medium         no  noauto

[138 rows x 7 columns]


# Pasos de la metodologìa STAR

## Particionar la muestra en casos positivos y negativos

In [8]:
casos_positivos = nuevo_dataset2[nuevo_dataset2["Class"] == 'auto']

casos_negativos = nuevo_dataset2[nuevo_dataset2['Class'] == 'noauto']


print("casos positivos: \n\n", casos_positivos)
print("casos negativos: \n\n", casos_negativos)

casos positivos: 

     Stability Error Sign  Wind   Magnitude Visibility Class
0       xstab    lx   pp  tail      medium        yes  auto
1        stab    ss   nn  head      strong        yes  auto
2        stab    ss   pp  tail      medium         no  auto
3        stab    lx   pp  head         low        yes  auto
4       xstab    xl   nn  head      strong        yes  auto
..        ...   ...  ...   ...         ...        ...   ...
123      stab    mm   pp  tail      strong        yes  auto
124     xstab    xl   nn  head         low        yes  auto
129      stab    ss   nn  head         low        yes  auto
133     xstab    lx   nn  head  OutOfRange        yes  auto
136     xstab    lx   pp  head         low        yes  auto

[95 rows x 7 columns]
casos negativos: 

     Stability Error Sign  Wind   Magnitude Visibility   Class
72      xstab    xl   nn  tail         low         no  noauto
75      xstab    ss   pp  tail  OutOfRange         no  noauto
76      xstab    ss   nn  head 

### Vamos a obtener todos los dominios de cada columna, porque los vamos a necesitar a continuacion

In [9]:
dominios_completos_atributos:list[list[str]] = []

for atributo in dataset.columns.to_list():

    dominios_completos_atributos.append(valores_observados(dataset[atributo].to_list()))

# Generamos las estrellas

In [10]:
from star import estrella, generarCubrimiento

estrellas:list[estrella] = []

estrellas = generarCubrimiento(estrellas, casos_positivos, casos_negativos, dominios_completos_atributos)

for una_estrella in estrellas:
    print(una_estrella.Su_Cubrimiento)

[['xstab', 'mm', 'pp', 'head', 'OutOfRange', 'yes']]
las combinaciones son: 

 [['xstab'], ['mm'], ['pp'], ['head'], ['OutOfRange'], ['yes'], ['xstab', 'mm'], ['xstab', 'pp'], ['xstab', 'head'], ['xstab', 'OutOfRange'], ['xstab', 'yes'], ['mm', 'pp'], ['mm', 'head'], ['mm', 'OutOfRange'], ['mm', 'yes'], ['pp', 'head'], ['pp', 'OutOfRange'], ['pp', 'yes'], ['head', 'OutOfRange'], ['head', 'yes'], ['OutOfRange', 'yes'], ['xstab', 'mm', 'pp'], ['xstab', 'mm', 'head'], ['xstab', 'mm', 'OutOfRange'], ['xstab', 'mm', 'yes'], ['xstab', 'pp', 'head'], ['xstab', 'pp', 'OutOfRange'], ['xstab', 'pp', 'yes'], ['xstab', 'head', 'OutOfRange'], ['xstab', 'head', 'yes'], ['xstab', 'OutOfRange', 'yes'], ['mm', 'pp', 'head'], ['mm', 'pp', 'OutOfRange'], ['mm', 'pp', 'yes'], ['mm', 'head', 'OutOfRange'], ['mm', 'head', 'yes'], ['mm', 'OutOfRange', 'yes'], ['pp', 'head', 'OutOfRange'], ['pp', 'head', 'yes'], ['pp', 'OutOfRange', 'yes'], ['head', 'OutOfRange', 'yes'], ['xstab', 'mm', 'pp', 'head'], ['xstab

# Ya tenemos las estrellas, lo que resta ahora es validar con el conjunto de prueba y generar las metricas del modelo aprendido

In [11]:
from star import prediccion_de_clase, probarHipotesis

prediccion = prediccion_de_clase(estrellas, test_set, clases)

print(len(prediccion))
print(len(test_set["Class"].values.tolist()))

140
140


In [12]:
probarHipotesis(prediccion, test_set["Class"].values.tolist(), [clases[0]])

vp: 42	fp: 4.0
fn: 4.0	 vn: 90
accuracy: 0.9428571428571428
precision: 0.9130434782608695
recall: 0.9130434782608695
f1: 0.9130434782608695


0

En la mayoría de las ejecuciones (personalmente no encontré ninguna que no fuera así) la matríz de confusión obtenida mostrará la diagonal de los falsos simétrica, es decir, que tanto fp como fn tendrán los mismos casos encontrados, esto es debido a que al generar nuestras estrellas, elminamos los complejos que cubren ejemplos negativos, es decir que nuestras estrellas son una solución consistente y no redundante. 

El problema está en que al generar las combinaciones para los valores *, hay duplicidad de registros que pertenecen a ambas clases auto y noauto, por lo que al eliminar los complejos que cubren casos negativos, se pierde tambien el complejo que cubre al caso positivo del registro duplicado.

Al no existir estrellas que cubran estos ejemplos, nuestro modelo aprendido clasifica como falsos negativos a los registros duplicados en ambas clases, pero son casos que no podemos aprender por falta de información para clasificarlos correctamente, por ello, esos casos duplicados se catalogan tanto positivos, como negativos, pero al ser un ejemplo que no predijo correctamente, se cataloga tanto como falso positivo como falso negativo.